In [1]:
from IPython.display import display

from typing import Dict

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# !pip install matplotlib

## Configs

In [3]:
nGPUs = (1,2,4)

## Read data

In [4]:
def read_csv(f_name: str) -> pd.DataFrame:
    try:
        _df = pd.read_csv(f_name, skiprows=3,engine='python', error_bad_lines=False)
        # print("%s UoM: "%f_name, _df.iloc[0])
        _df = _df.drop(0)
        _df = _df.astype({'Start': float, 'Duration': float})
    except:
        _df = pd.read_csv(f_name, skiprows=3, skipfooter=1)
        # print("%s UoM: "%f_name, _df.iloc[0])
        _df = _df.drop(0)
        _df = _df.astype({'Start': float, 'Duration': float})
    return _df

def get_f_name(n_gpu: int) -> str:
    return "profiler_gpt2yelp_bs4_gpu%d.csv"%n_gpu



## Analysis

In [5]:
def get_ins_group(ops: str) -> str:
    if "gemm" in ops:
        return "matrix-mul"
    elif "CUDA memcpy" in ops or "nccl" in ops.lower() or "copy_device_to_device" in ops.lower()\
        or "CUDA memset" in ops:
        return "memory_mgmt"
    elif "::native" in ops or "vectorized_elementwise" in ops or "_cpp1_ii" in ops or "reduce_kernel" in ops:
        return "custom_ops"
    return "other"

def aggregate(df: pd.DataFrame) -> None:
    df["ops_group"] = df.apply(lambda x: get_ins_group(x.Name), axis=1)
    
    called_cuda_interfaces = df["Name"].unique().tolist()
    memory_interfaces = list(filter(lambda x: 'CUDA mem' in x, called_cuda_interfaces))
    memory_interfaces.append('ncclBroadcastRingLLKernel_copy_i8(ncclColl)')
    # Elapsed time
    print("--------- Elapsed time (ms): ", df["Start"].max() - df["Start"].min())
    # devices
    print("--------- devices ")
    display(df["Device"].unique().tolist())
    # memory mgmt
    print("--------- memory mgmt ")
    for mi in memory_interfaces:
        print(mi)
        display(df.query("Name == '%s'"%mi).groupby('Device')["Duration"].sum().reset_index())
    display(df[df["Name"].isin(memory_interfaces)].groupby('Device')["Duration"].sum().reset_index())
    # Top N
    print("--------- Top N ")
    display(df.groupby(['Device', 'Name'])["Duration"].sum().nlargest(10).reset_index())
    display(df.groupby(['Device', 'ops_group'])["Duration"].sum().reset_index())
    # Aggregate
    print("--------- Aggregate ")
    agg_result = df.groupby('ops_group')["Duration"].sum().reset_index()
    display(agg_result)
    agg_result["Percent"] = agg_result[["Duration",]].apply(lambda x: 100*x/x.sum())
    display(agg_result)

def analyze(n_gpus: int = nGPUs) -> None:
    for n_gpu in n_gpus:
        print("=========== Start %d GPUs ==========="%n_gpu)
        f_name = get_f_name(n_gpu)
        df = read_csv("profiler_gpt2yelp_bs4_gpu%d.csv"%n_gpu)
        aggregate(df)
        print("=========== End %d GPUs =========== \n\n\n"%n_gpu)

## Check Unique GPUs

In [6]:
analyze((1,2,4))

=========== Start 1 GPUs ===========


Skipping line 67450: unexpected end of data


--------- Elapsed time (ms):  16.363323
--------- devices 


['Tesla V100-SXM2-32GB-LS (0)']

--------- memory mgmt 
[CUDA memcpy HtoD]


...

,Device,ops_group,Duration
0,Tesla V100-SXM2-32GB-LS (0),custom_ops,10293.078968
1,Tesla V100-SXM2-32GB-LS (0),matrix-mul,14621.977590
2,Tesla V100-SXM2-32GB-LS (0),memory_mgmt,13381.122406
3,Tesla V100-SXM2-32GB-LS (0),other,18.584924
4,Tesla V100-SXM2-32GB-LS (1),custom_ops,9246.981013
5,Tesla V100-SXM2-32GB-LS (1),matrix-mul,14896.915461
6,Tesla V100-SXM2-32GB-LS (1),memory_mgmt,10554.359265
7,Tesla V100-SXM2-32GB-LS (1),other,17.572509
8,Tesla V100-SXM2-32GB-LS (2),custom_ops,8509.420120
9,Tesla V100-SXM2-32GB-LS (2),matrix-mul,14424.244212


--------- Aggregate 


,ops_group,Duration
0,custom_ops,36693.225301
1,matrix-mul,57910.628717
2,memory_mgmt,47659.894966
3,other,72.240242


,ops_group,Duration,Percent
0,custom_ops,36693.225301,25.779303
1,matrix-mul,57910.628717,40.685865
2,memory_mgmt,47659.894966,33.484079
3,other,72.240242,0.050753


=========== End 4 GPUs =========== 



